In [7]:
#Esta guía explica el proceso para crear una arquitectura de redes neuronales, posteriormente explica el proceso de 
#optimización y fine tuning. Para ello, primeramente se define una red neuronal y se evalúa. Posteriormente, se ajustan
#los parámetros y se evalúa el modelo final.

#El dataset se llama Titanic, lo pueden encontrar en: 
#Dataset Kaggle: https://www.kaggle.com/c/titanic
#También lo adjunto en el directorio de esta guía

#Con este problema se intenta predecir si una persona sobrevivió o no sobrevivió, este tipo de problemas es aplicable a otros 
#problemas de sobrevivencia ante desastres.
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

df = pd.read_csv('bank-full.csv',sep=';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [8]:
df = df.dropna(subset=["age", 'job', 'default', 'balance', 'housing','loan','duration','campaign','previous', 'poutcome','y'])
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [9]:
#Codificar datos de salida
import numpy as np
from sklearn import preprocessing
marital_status = [];
columna_marital_status = df.iloc[:,16];
marital_status = np.array(columna_marital_status)

# Etiquetas de entrada de muestra
marital_status

# Crear codificador de etiquetas y ajustar las etiquetas
encoder = preprocessing.LabelEncoder()
encoder.fit(marital_status)

# Imprimir el mapeo
print("\nLabel mapping:")
for i, item in enumerate(encoder.classes_):
    print(item, '-->', i)
    
#print('Prueba');
#print(y)
# Codificar un conjunto de etiquetas usando el codificador 
test_labels2 = marital_status
encoded_values2 = encoder.transform(test_labels2)
#print("\nLabels =", test_labels2)
#print("Encoded values =", list(encoded_values2))

df['y']=encoded_values2;
df


Label mapping:
no --> 0
yes --> 1


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,1
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,1
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,0


In [4]:
#Seleccionamos las variables escogidas

Xsubset = df[["age", 'job', 'default', 'balance', 'housing','loan','duration','campaign','previous', 'poutcome']]

#para separar nuestra variable dependiente de la independiente, haremos lo siguiente:

y = df.y.values
print(Xsubset)
#print(y)
type(Xsubset)

       age           job default  balance housing loan  duration  campaign  \
0       58    management      no     2143     yes   no       261         1   
1       44    technician      no       29     yes   no       151         1   
2       33  entrepreneur      no        2     yes  yes        76         1   
3       47   blue-collar      no     1506     yes   no        92         1   
4       33       unknown      no        1      no   no       198         1   
...    ...           ...     ...      ...     ...  ...       ...       ...   
45206   51    technician      no      825      no   no       977         3   
45207   71       retired      no     1729      no   no       456         2   
45208   72       retired      no     5715      no   no      1127         5   
45209   57   blue-collar      no      668      no   no       508         4   
45210   37  entrepreneur      no     2971      no   no       361         2   

       previous poutcome  
0             0  unknown  
1        

pandas.core.frame.DataFrame

In [10]:
#SEGUNDA FORMA DE PREPROCESAR

#--------Variables a normalizar---
numeric_features = ['age','balance','duration','campaign','previous']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])


#-----------Variables categoricas nominales--------

categorical_features = ['job','default','housing','loan','poutcome']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


clf = Pipeline(steps=[('preprocessor', preprocessor)])

print(clf)

X = clf.fit_transform(Xsubset)
print(X.shape[1])
print(X.shape)
print(X)
#print(clf)

cnamesDataset1 = ['age','balance','duration','campaign','previous']
cnamesDataset2 = clf.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['onehot'].get_feature_names(categorical_features)


cnamesDataset1.extend(cnamesDataset2)

DatasetPreprocesado = pd.DataFrame(data=X,columns=cnamesDataset1)
print(DatasetPreprocesado.head())
DatasetPreprocesado.to_csv("DatasetPreprocesado.csv", sep=";",index = False) #sep es el separado, por defector es ","



Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['age', 'balance', 'duration',
                                                   'campaign', 'previous']),
                               

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=10)

#Ahora preparamos el perceptron. Importamos las neuronas simples y el modelo secuencial
#Modelo secuencial quiere decir que agregaremos capas y se conectarán de manera automática, 
#Dense es la librería de neuronas simples.
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import model_from_json
print('Librerías importadas')

Librerías importadas


In [13]:
X_train.shape[1]
#X_train
y_train


array([0, 0, 0, ..., 0, 1, 0])

In [14]:
#FUNCIONES PARA GuARDAR Y CARGAR CUALQUIER MODELO

#Guardar pesos y la arquitectura de la red en un archivo 

def guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos):
    print("Guardando Red Neuronal en Archivo")  
    # serializar modelo a JSON

    # Guardar los Pesos (weights)
    model.save_weights(nombreArchivoPesos+'.h5')

    # Guardar la Arquitectura del modelo
    with open(nombreArchivoModelo+'.json', 'w') as f:
        f.write(model.to_json())

    print("Red Neuronal Grabada en Archivo")   
    
def cargarRNN(nombreArchivoModelo,nombreArchivoPesos):
        
    # Cargar la Arquitectura desde el archivo JSON
    with open(nombreArchivoModelo+'.json', 'r') as f:
        model = model_from_json(f.read())

    # Cargar Pesos (weights) en el nuevo modelo
    model.load_weights(nombreArchivoPesos+'.h5')  

    print("Red Neuronal Cargada desde Archivo") 
    return model

In [15]:
#Construcción del Modelo o Arquitectura de Redes Neoronales
model = Sequential()

#La primera capa Dense recibe el numero de variables, que es la segunda dimensión de la matriz X, esto es X_train.shape[1]
#La primera capa tiene 32 neuronas. La función de activación es la función rectificadora.
model.add(Dense(27, input_shape=(X_train.shape[1],), activation='relu'))
#La segunda capa tiene 64 neuronas. La función de activación es la función rectificadora.
model.add(Dense(10, activation='relu'))
#La capa de salida tiene 1 neurona. La capa de salida debe tener la misma dimensión como de cantidad de salidas queremos,
#por ejemplo, en este caso la salida "Survived" solo requiere 0 y 1. Puesto que 0 o 1 ocupan solo un valor dentro de cada dato,
#entonces 1 neurona es suficiente. La función de activación es sigmoide para clasificación por probabilidad.
model.add(Dense(1, activation='sigmoid'))

#Como tenemos dos posibles salidas "0 o 1", vamos a escoger que el error lo trate como una clasificación binaria, 
#el optimizador será nuestra función derivada que nos ayudará a determinar hacia donde mover los pesos.
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method

#imprimir arquitectura de la red
model.summary()

#Entrenamiento: 

#Entrenaremos por 100 epochs, el batch_size es un argumento importante, porque representa cada cuántos datos va a actualizar
#los pesos. Este es el método del gradiente descendiente estocástico que hace el proceso más eficiente y preciso.
model.fit(X_train, y_train, epochs=50, batch_size=100, verbose=0)
#print(model.fit(X_train, y_train, epochs=50, batch_size=100, verbose=0))

score = model.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
score = model.evaluate(X_test, y_test, verbose=0)
print('Resultado en Test:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#mostrar pesos de la red
#print(model.get_weights())

#Guardar pesos y la arquitectura de la red en un archivo 

nombreArchivoModelo='arquitectura_prueba'
nombreArchivoPesos='pesos_prueba'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

#Cargar pesos y la arquitectura
model2=cargarRNN(nombreArchivoModelo,nombreArchivoPesos) 

model2.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method
score = model2.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model2.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 27)                756       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                280       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 1,047
Trainable params: 1,047
Non-trainable params: 0
_________________________________________________________________
Resultado en Train:
acc: 90.60%
Resultado en Test:
acc: 89.88%
Guardando Red Neuronal en Archivo
Red Neuronal Grabada en Archivo
Red Neuronal Cargada desde Archivo
Resultado en Train:
acc: 90.60%
Resultado en Test:
acc: 89.88%


In [17]:
#importamos el algoritmo cross validator, y un wrapper que permitirá usar modelos de keras con scikit learn
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
print('Librerías importadas')

Librerías importadas


In [ ]:
#2. Densidad de las capas de neuronas
#Notemos que se incluyen los mejores parámetros del paso de optimización anterior (batch_size, epochs y optimizer)
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(l1, l2):
    model = Sequential()
    model.add(Dense(l1, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'l1':[16,32,64,128,256],
                           'l2':[16,23,64,128,256]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)

#Resultados: {'l1': 32, 'l2': 16}
#Los resultados indican que hubo un error en la red original, las capas van desde la más densa, a la menos densa.